# Evaluation of experiments - E1 COGNITION
fistly define the UUID of the experiment to use

In [1]:
experimentId="f7a9e88f-5e0f-4465-a5ff-1af9e2f93866"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load cognition

In [3]:
samplerate=200  #Should rethink this
inputDimension=601

In [4]:
signals, annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath,returnSignals=True)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

2785

In [6]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Preprocess

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [7]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [8]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## prepare slides and labels for training

In [9]:
signalsMetadata['excerptDimension']=signalsMetadata.apply(lambda row: int(row.duration*row.samplerate),axis=1)
allExtended=pd.DataFrame()
for ind,row in signalsMetadata.iterrows():
    thisExtended=pd.DataFrame({
        'trainIndex':np.arange(row.excerptDimension),
        'subjectId':row.subjectId
    })
    allExtended=pd.concat([allExtended,thisExtended])

In [10]:
usedAnnotations=annotations[annotations.labelerId=='0001'].reset_index(drop=True)

In [11]:
labels={}
for index, row in signalsMetadata.iterrows():
    labels[row.subjectId]=excerptAnnotationsToLabels(usedAnnotations[usedAnnotations.subjectId==row.subjectId],excerptDimension=row.excerptDimension)

In [12]:
paddedSignals={}
for index, row in signalsMetadata.iterrows():
    paddedSignals[row.subjectId]=padVectorBothSides(signals[row.subjectId],window2half(inputDimension),method='closest')

In [13]:
slides={}
for index, row in signalsMetadata.iterrows():
    slides[row.subjectId]=np_tricks.sliding_window_view(paddedSignals[row.subjectId],(inputDimension,))

## Load experiment results

In [14]:
experimentModels = loadExperiment(experimentId,datapath)

In [15]:
experimentModels.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,ca4abebe-7534-4387-9f7e-7e56aeb102df
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,ec345db7-d0e7-447a-a219-52693d52bf41
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,5efd55af-ba1f-4dd8-905e-9279a3c25d07
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,79b2270a-cfb8-4f96-987a-040fa3dc0386
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,b1cb5f16-0c62-4dd7-b5d9-16fae63b28c5


## Optimal hyperparameter estimation with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [16]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,ca4abebe-7534-4387-9f7e-7e56aeb102df
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,ec345db7-d0e7-447a-a219-52693d52bf41
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,5efd55af-ba1f-4dd8-905e-9279a3c25d07
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,79b2270a-cfb8-4f96-987a-040fa3dc0386
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,b1cb5f16-0c62-4dd7-b5d9-16fae63b28c5
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,28b0a080-fdd1-4b49-97d4-f98a8671c508
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,acc1aab9-4594-40ce-8d0c-f2111f7a6c69


In [17]:
auxThres=pd.DataFrame({
    'hyperThres':[0.1,0.2,0.3,0.4,0.5,0.6]
})
auxMinDuration=pd.DataFrame({
    'hyperMinDuration':[0.3]
})
""" auxMaxDuration=pd.DataFrame({
    'hyperMaxDuration':[5]
}) """
auxClose=pd.DataFrame({
    'hyperClose':[0.1]
})


hyperParams=pd.merge(auxThres,auxMinDuration,how='cross')
""" hyperParams=pd.merge(hyperParams,auxMaxDuration,how='cross') """
hyperParams=pd.merge(hyperParams,auxClose,how='cross')

hyperParams

,hyperThres,hyperMinDuration,hyperClose
0,0.1,0.3,0.1
1,0.2,0.3,0.1
2,0.3,0.3,0.1
3,0.4,0.3,0.1
4,0.5,0.3,0.1
5,0.6,0.3,0.1


In [18]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperMinDuration=row_hyper.hyperMinDuration
    hyperClose=row_hyper.hyperClose
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadDL(row.modelId,experimentId,datapath)
        #initialise lists
        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            inferred=model.predict(paddedSignals[valSubjectId][np.newaxis,...,np.newaxis],verbose=0)
            rawLabels=inferred>=hyperThres
            processedLabels=labelingProcess(rawLabels.flatten(),hyperClose,hyperMinDuration,samplerate)
            processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
            gtAnnotations=labelVectorToAnnotations(labels[valSubjectId],samplerate) 
            f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

0
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-09-01 14:27:40         5064


metadata.json                                  2023-09-01 14:27:40           64
variables.h5                                   2023-09-01 14:27:40      1800912
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\conv1d_2
......vars
.........0
.........1
...layers\conv1d_3
......vars
.........0
.........1
...layers\conv1d_4
......vars
.........0
.........1
...layers\conv1d_5
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive loading:
Fi

In [19]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [20]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

maximal mean score at:


hyperThres          0.400000
hyperMinDuration    0.300000
hyperClose          0.100000
meanMeanF1          0.725061
stdMeanF1           0.043897
Name: 3, dtype: float64

## Graphical representation of the optimal point

In [22]:
aux=hyperParams[(hyperParams.hyperMinDuration==optimal.hyperMinDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')